Написать краулер, который собирает тексты с новостного ресурса.


In [1]:
import requests
from bs4 import BeautifulSoup
from IPython.display import HTML, display
import re

In [2]:
url = 'http://susnov.ru'

In [3]:
def main_pages(url):
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'lxml')
    links = []
    for h in soup.findAll(attrs={'class' : re.compile("iceMenuTitle$")}):
        if 'http' not in h['href']:
            links.append(url + h['href'])
        else:
            links.append(h['href'])

    # почему-то в меню не все подкатегории, добираем их
    extra_links = []
    for link in links:
        req = requests.get(link)
        soup = BeautifulSoup(req.text, 'lxml')
        for h in soup.findAll(attrs={'class' : "cat-children"}):
            a = h.findAll('a')
            for i in a:
                if ':' not in i['href']: # убираем ссылки на другие сайты и прочий мусор
                    extra_links.append(url + i['href'])
    links = set(links)
    links |= set(extra_links)
    return links

In [52]:
main = main_pages(url)
main

{'http://susnov.ru/',
 'http://susnov.ru/glavnaya/ekonomika.html',
 'http://susnov.ru/glavnaya/kultura-turizm-molodjozhnaya-politika.html',
 'http://susnov.ru/glavnaya/obrazovanie.html',
 'http://susnov.ru/glavnaya/pravoporyadok.html',
 'http://susnov.ru/glavnaya/proisshestviya.html',
 'http://susnov.ru/glavnaya/sport.html',
 'http://susnov.ru/glavnaya/v-rajonakh-oblasti.html',
 'http://susnov.ru/glavnaya/vlast-i-obshchestvo.html',
 'http://susnov.ru/glavnaya/zdravookhranenie.html',
 'http://susnov.ru/k-75-letiyu-pobedy.html',
 'http://susnov.ru/k-75-letiyu-pobedy/altar-pobedy.html',
 'http://susnov.ru/k-75-letiyu-pobedy/khroniki-pobedy.html',
 'http://susnov.ru/k-75-letiyu-pobedy/urok-muzhestva.html',
 'http://susnov.ru/o-gazete.html',
 'http://susnov.ru/o-gazete/prajs.html',
 'http://susnov.ru/rasskazyvaet-obshchestvennyj-korrespondent.html',
 'http://susnov.ru/rasskazyvaet-obshchestvennyj-korrespondent/kudryavtsev-valentin-nikiforovich.html',
 'http://susnov.ru/rasskazyvaet-obshches

In [4]:
# собираем ссылки на статьи с данной страницы 
def take_links(url, soup):
    links = []
    for h in soup.findAll(attrs={'class' : 'blog blog'}):
        a = h.findAll('a', title = False)
        for i in a:
            if ':' not in i['href']: 
                links.append(url + i['href'])
            
    return links

In [5]:
def all_article_links(url):
    links = []
    main = main_pages(url)
    
    for link in main:
        req = requests.get(link)
        soup = BeautifulSoup(req.text, 'lxml')
        links += take_links(url, soup)
        try:
            pages = int(soup.find('a', {'title':'В конец'})['href'].split('=')[1])
            for n in range(7, pages, 7):
                req = requests.get(link+'?start='+str(n))
                soup = BeautifulSoup(req.text, 'lxml')
                links += take_links(url, soup)
        except:
            pass
    links = set(links)
    links -= set(main)
    return links

In [7]:
def write_articles(url):
    article_links = all_article_links(url)
    print('Всего статей: ', len(article_links))
    for link in article_links:
        req = requests.get(link)
        soup = BeautifulSoup(req.text, 'lxml')
        author = soup.find('strong').text.split('. ')[0]
        title = soup.find('h2').text.strip()
        date = soup.find('dd', attrs = {'class': 'published'}).text.split()[1]
        # есть статьи с неуказанной категорией
        try:
            topic = soup.find('dd', attrs = {'class': 'category-name'}).find('a').text
        except:
            topic = 'None'
        # иногда текст статьи находится вне тегов <p></p>
        try:
            text = soup.find('div', attrs = {'class':'item-page item-page'}).findAll('p')
            text = ' '.join([t.text for t in text])
        except AttributeError:
            text = soup.find('div', attrs = {'class':'item-page item-page'}).text
        # а тут ругается на длинные названия статей == txt файлов
        try:
            with open(title+'.txt', 'w') as file:
                file.write('@au '+author+'\n@ti '+title+'\n@da '+date+'\n@topic '+topic+'\n@url '+link+'\n'+text)
        except OSError:
            title = title[:40]
            with open(title+'.txt', 'w') as file:
                file.write('@au '+author+'\n@ti '+title+'\n@da '+date+'\n@topic '+topic+'\n@url '+link+'\n'+text)

In [8]:
write_articles(url)

Всего статей:  4977
